<a href="https://colab.research.google.com/github/horsinnaround/hagging-fuckin-face/blob/main/01_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Uncomment and run this cell if you're on Colab or Kaggle
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements()

Cloning into 'notebooks'...
remote: Enumerating objects: 507, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 507 (delta 100), reused 100 (delta 87), pack-reused 378
Receiving objects: 100% (507/507), 30.39 MiB | 17.24 MiB/s, done.
Resolving deltas: 100% (236/236), done.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


In [2]:
#hide
from utils import *
setup_chapter()

No GPU was detected! This notebook can be *very* slow without a GPU 🐢
Go to Runtime > Change runtime type and select a GPU hardware accelerator.
Using transformers v4.11.3
Using datasets v1.16.1


# Hello Transformers

<img alt="transformer-timeline" caption="The transformers timeline" src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter01_timeline.png?raw=1" id="transformer-timeline"/>

## The Encoder-Decoder Framework

<img alt="rnn" caption="Unrolling an RNN in time." src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter01_rnn.png?raw=1" id="rnn"/>

<img alt="enc-dec" caption="Encoder-decoder architecture with a pair of RNNs. In general, there are many more recurrent layers than those shown." src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter01_enc-dec.png?raw=1" id="enc-dec"/>

## Attention Mechanisms

<img alt="enc-dec-attn" caption="Encoder-decoder architecture with an attention mechanism for a pair of RNNs." src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter01_enc-dec-attn.png?raw=1" id="enc-dec-attn"/> 

<img alt="attention-alignment" width="500" caption="RNN encoder-decoder alignment of words in English and the generated translation in French (courtesy of Dzmitry Bahdanau)." src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter02_attention-alignment.png?raw=1" id="attention-alignment"/> 

<img alt="transformer-self-attn" caption="Encoder-decoder architecture of the original Transformer." src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter01_self-attention.png?raw=1" id="transformer-self-attn"/> 

## Transfer Learning in NLP

<img alt="transfer-learning" caption="Comparison of traditional supervised learning (left) and transfer learning (right)." src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter01_transfer-learning.png?raw=1" id="transfer-learning"/>  

<img alt="ulmfit" width="500" caption="The ULMFiT process (courtesy of Jeremy Howard)." src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter01_ulmfit.png?raw=1" id="ulmfit"/>

## Hugging Face Transformers: Bridging the Gap

## A Tour of Transformer Applications

In [3]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

### Text Classification

In [4]:
#hide_output
from transformers import pipeline

classifier = pipeline("text-classification")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [10]:
import pandas as pd
text = 'the tea is too hot'
outputs = classifier(text)
pd.DataFrame(outputs)    

,label,score
0,NEGATIVE,0.996909


In [11]:
outputs[0]['score']

0.9969093203544617

In [26]:
def trans(text):
    outputs = classifier(text)
    return outputs[0]['score'],outputs[0]['label']

In [117]:
trans("im so and grateful now that affirmations")

(0.9998443126678467, 'POSITIVE')

In [131]:
train_data = '/content/drive/MyDrive/work 13 Spam Email Deep Learning/train_E6oV3lV.csv'
# test_data = '/content/drive/MyDrive/work 13 Spam Email Deep Learning/test_tweets_anuFYb8.csv'
dtrain = pd.read_csv(train_data)
# dtest = pd.read_csv(test_data)
dtrain.drop('id',axis=1,inplace=True)
# dtest.drop('id',axis=1,inplace=True)
# import re
# def  clean_text(df, text_field):
#     df[text_field] = df[text_field].str.lower()
#     df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
#     df[text_field] = df[text_field].apply(lambda elem: re.sub(r"([^A-Za-z\t])", " ", elem))  
#     df[text_field] = df[text_field].apply(lambda elem: re.sub(r"  ", " ", elem))  
#     df[text_field] = df[text_field].apply(lambda elem: re.sub(r"  ", " ", elem)) 
#     df[text_field] = df[text_field].apply(lambda elem: re.sub(r"  ", " ", elem))  
#     df[text_field] = df[text_field].apply(lambda elem: re.sub(r"  ", " ", elem))  



    
#     return df
# trainc = clean_text(dtrain,'tweet')
# # testc = clean_text(dtest,'tweet')
# trainc

In [327]:
p = 'let us fight against love'
p,trans(p)

# trans('')

('let us fight against love', (0.9982835054397583, 'POSITIVE'))

In [114]:
h1 = trainc['tweet'].apply(lambda text:trans(text))
h1

0        (0.9989578723907471, NEGATIVE)
1        (0.9751367568969727, NEGATIVE)
2        (0.9879195690155029, POSITIVE)
3        (0.9975787997245789, POSITIVE)
4        (0.7010807991027832, POSITIVE)
                      ...              
31957    (0.9611828327178955, NEGATIVE)
31958    (0.9867665767669678, NEGATIVE)
31959    (0.9993545413017273, NEGATIVE)
31960    (0.9979729056358337, NEGATIVE)
31961    (0.9998412132263184, POSITIVE)
Name: tweet, Length: 31962, dtype: object

In [121]:
lab = []
pos = []
for i in h1:
    lab.append(i[0])
    pos.append(i[1])

In [125]:
dtrain['lab'] = lab
dtrain['pos'] = pos
trainc['lab'] = lab
trainc['pos'] = pos

In [126]:
trainc.iloc[31954].tweet

'holiday bull up you will dominate your bull and you will direct it whatever you want it to do when you '

In [338]:
tt = trainc.drop(trainc[trainc['lab']<0.50].index)

In [356]:
tts = tt.drop(tt[(tt['pos']=="NEGATIVE") & (tt['label']==0)].index)

In [351]:
tts = tts.drop(tts[(tts['pos']=="POSITIVE") & (tts['label']==1)].index)


In [261]:
tt.loc[31958].tweet

' to see nina turner on the airwaves trying to wrap herself in the mantle of a genuine hero like shirley chisolm shame imwithher'

In [21]:
trainc.trans = trainc.trans.map({0:1,1:0})

In [265]:
path = '/content/drive/MyDrive/work 13 Hateful Speach Deep Learning/trainc3.csv'
# os.mkdir(path)
trainc.to_csv(path,index=False)

In [24]:
trainc.trans.value_counts()

0    17736
1    14226
Name: trans, dtype: int64

In [25]:
trainc.label.value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [366]:
# tt
import numpy as np
dtr = tts.sample(frac=0.8,random_state=0)
dts = tts.drop(dtr.index)
print(f"Len  train {dtr.shape[0]} and len tes {dts.shape[0]}")
from tensorflow.keras.layers import TextVectorization
TVT = TextVectorization(
    max_tokens=4000,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    ngrams=1,
    output_mode="int",
    output_sequence_length=100,
    pad_to_max_tokens=True,
    vocabulary=None,

)
from pandas._libs.lib import dicts_to_array
TVT.adapt(dtr.tweet)
trainpadded = TVT(dtr.tweet)
testpadded = TVT(dts.tweet)
TVT.vocabulary_size()
X = np.array(trainpadded)
x = np.array(testpadded)
Y = dtr['label'].to_numpy()
y = dts['label'].to_numpy()

# Modeling
from keras.models import Sequential
from keras.layers import *
from keras import layers
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

# Define parameter
vocab_size = 4000+1
embedding_dim = 40
drop_value = 0.2
n_dense = 40
max_len = 100

model = tf.keras.Sequential([
    Embedding(vocab_size,
                    embedding_dim,
                    input_length = max_len),
    Conv1D(32, 9, activation='relu'),
    # Conv1D(32, 6, activation='relu'),
    AveragePooling1D(),
    Conv1D(32, 4, activation='relu'),
    # Conv1D(32, 5, activation='relu'),
    # Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
    # Bidirectional(tf.keras.layers.LSTM(24)),
    # Bidirectional(tf.keras.layers.LSTM(24,return_sequences=True)),
    
    # Attention(units=128),

    # GlobalAveragePooling1D(),
    # GlobalMaxPooling1D(),
    # AveragePooling1D(),
    Flatten(),

    # Dropout(0.25),
    Dropout(0.5),
    Dense(32 ,activation='relu'
    ,
                          kernel_regularizer = tf.keras.regularizers.l2(0.001)
                         ),
    Dropout(0.5),
    

    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam'
              ,loss='binary_crossentropy',
              metrics=[
                  tf.keras.metrics.Recall()
                ,tf.keras.metrics.Precision(),
                # 'accuracy'
            # ,
            # tf.keras.metrics.AUC()
              ])
model.summary()

Len  train 14500 and len tes 3625
Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, 100, 40)           160040    
                                                                 
 conv1d_8 (Conv1D)           (None, 92, 32)            11552     
                                                                 
 average_pooling1d (AverageP  (None, 46, 32)           0         
 ooling1D)                                                       
                                                                 
 conv1d_9 (Conv1D)           (None, 43, 32)            4128      
                                                                 
 flatten_1 (Flatten)         (None, 1376)              0         
                                                                 
 dropout_23 (Dropout)        (None, 1376)              0         
                   

In [370]:
88 * 97 * 2 / (88 + 97)

92.28108108108108

In [367]:
num_epochs = 12
early_stop = EarlyStopping(monitor='val_loss', patience=4,restore_best_weights=True)
history = model.fit(X,
                    Y,
                    batch_size=32,
                    epochs=num_epochs, 
                    validation_data=(x, y),
                    # callbacks =[early_stop],
                    )

Epoch 1/12
454/454 [==============================] - 11s 22ms/step - loss: 0.2737 - recall_12: 0.2891 - precision_2: 0.7631 - val_loss: 0.1372 - val_recall_12: 0.7770 - val_precision_2: 0.8509
Epoch 2/12
454/454 [==============================] - 10s 21ms/step - loss: 0.1234 - recall_12: 0.8249 - precision_2: 0.8853 - val_loss: 0.1282 - val_recall_12: 0.7746 - val_precision_2: 0.8943
Epoch 3/12
454/454 [==============================] - 10s 21ms/step - loss: 0.0870 - recall_12: 0.8888 - precision_2: 0.9239 - val_loss: 0.1278 - val_recall_12: 0.8122 - val_precision_2: 0.8543
Epoch 4/12
454/454 [==============================] - 10s 21ms/step - loss: 0.0620 - recall_12: 0.9301 - precision_2: 0.9526 - val_loss: 0.1586 - val_recall_12: 0.7817 - val_precision_2: 0.8928
Epoch 5/12
454/454 [==============================] - 10s 21ms/step - loss: 0.0502 - recall_12: 0.9548 - precision_2: 0.9644 - val_loss: 0.2108 - val_recall_12: 0.7700 - val_precision_2: 0.8937
Epoch 6/12
454/454 [==========

In [368]:
tts.label.value_counts()

0    15883
1     2242
Name: label, dtype: int64

In [369]:
y_ = model.predict(x)
tr = 0.5
y_[y_>tr] = 1
y_[y_<=tr] = 0
print(classification_report(y,y_))

114/114 [==============================] - 1s 5ms/step
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      3199
           1       0.81      0.82      0.81       426

    accuracy                           0.96      3625
   macro avg       0.89      0.90      0.89      3625
weighted avg       0.96      0.96      0.96      3625



### Named Entity Recognition

In [ ]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)    

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556569,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498350,##icons,259,264
7,MISC,0.775361,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


### Question Answering 

In [ ]:
reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])    

,score,start,end,answer
0,0.631291,335,358,an exchange of Megatron


### Summarization

In [ ]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

 Bumblebee ordered an Optimus Prime action figure from your online store in
Germany. Unfortunately, when I opened the package, I discovered to my horror
that I had been sent an action figure of Megatron instead.


### Translation

In [ ]:
translator = pipeline("translation_en_to_de", 
                      model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus
Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete,
entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von
Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich
hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere
einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt.
Anbei sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, bald von
Ihnen zu hören. Aufrichtig, Bumblebee.


### Text Generation

In [ ]:
#hide
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results

In [ ]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

Dear Amazon, last week I ordered an Optimus Prime action figure from your online
store in Germany. Unfortunately, when I opened the package, I discovered to my
horror that I had been sent an action figure of Megatron instead! As a lifelong
enemy of the Decepticons, I hope you can understand my dilemma. To resolve the
issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered.
Enclosed are copies of my records concerning this purchase. I expect to hear
from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. The order was
completely mislabeled, which is very common in our online store, but I can
appreciate it because it was my understanding from this site and our customer
service of the previous day that your order was not made correct in our mind and
that we are in a process of resolving this matter. We can assure you that your
order


## The Hugging Face Ecosystem

<img alt="ecosystem" width="500" caption="An overview of the Hugging Face ecosystem of libraries and the Hub." src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter01_hf-ecosystem.png?raw=1" id="ecosystem"/>

### The Hugging Face Hub

<img alt="hub-overview" width="1000" caption="The models page of the Hugging Face Hub, showing filters on the left and a list of models on the right." src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter01_hub-overview.png?raw=1" id="hub-overview"/> 

<img alt="hub-model-card" width="1000" caption="A example model card from the Hugging Face Hub. The inference widget is shown on the right, where you can interact with the model." src="https://github.com/nlp-with-transformers/notebooks/blob/main/images/chapter01_hub-model-card.png?raw=1" id="hub-model-card"/> 

### Hugging Face Tokenizers

### Hugging Face Datasets

### Hugging Face Accelerate

## Main Challenges with Transformers

## Conclusion